In [1]:
dataset = 'dbbook'

domain = {
    'lastfm': 'music',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]

items = {
    'lastfm': 'artist',
    'dbbook': 'book',
    'movielens': 'movie'
}[dataset]
import os
datapath = os.path.join('..', 'data', dataset)

special_token = '<|reserved_special_token_22|>'

resp_frac = 0.5

# Set names

In [2]:
import pandas as pd
import os
import random
import numpy as np
import swifter  

In [3]:
user_item_df_train = pd.read_csv(os.path.join(datapath, 'user-item', 'train.tsv'), \
            names=['user', 'item', 'rating'], sep='\t')

user_item_df_test = pd.read_csv(os.path.join(datapath, 'user-item', 'test.tsv'), \
            names=['user', 'item', 'rating'], sep='\t')


df = pd.concat([user_item_df_train, user_item_df_test], ignore_index=True)
all_items = df['item'].unique()

In [4]:
def get_split_random(list_to_split, frac=0.2, margin=0):
    random.seed(22)
    sample_list = random.sample(list_to_split, int(frac*len(list_to_split))+margin)
    complementary_set = set(list_to_split) - set(sample_list)
    return list(set(complementary_set)), list(set(sample_list))

In [5]:
random.seed(22)
test_frac = 0.2
user_list = df['user'].unique().tolist()
user_train, user_test = get_split_random(user_list, frac=test_frac, margin=0)

In [6]:
user_item_df_train = df[df['user'].isin(user_train)]
user_item_df_test = df[df['user'].isin(user_test)]

In [ ]:
len(set(user_item_df_train['user'].unique()) - set(user_item_df_test['user'].unique()))

In [ ]:
user_item_df_test['rating'].describe()

In [ ]:
user_item_df_train['rating'].describe()

In [10]:
train_group = user_item_df_train.groupby('user').agg({
    'item': list,
    'rating': list
}).reset_index()
test_group = user_item_df_test.groupby('user').agg({
    'item': list,
    'rating': list
}).reset_index()
train_group['len'] = train_group['item'].apply(len)
test_group['len'] = test_group['item'].apply(len)

In [11]:
def split_prompt_resp(row, resp_frac):
    items = np.array(row['rating'])
    index_likes = np.where(items==1)[0].tolist()
    index_dilikes = np.where(items==0)[0].tolist()
    index_like_prompt = []
    index_like_resp = []    
    index_dislike_prompt = []
    index_dislike_resp = []
    resp_frac=resp_frac
    if len(index_likes) > 0:
        index_like_prompt, index_like_resp = get_split_random(index_likes, frac=resp_frac, margin=1)
    if len(index_dilikes) > 0:
        index_dislike_prompt, index_dislike_resp = get_split_random(index_dilikes, frac=resp_frac, margin=1)
    row['index_like_prompt'] = index_like_prompt
    row['index_like_resp'] = index_like_resp
    row['index_dislike_prompt'] = index_dislike_prompt
    row['index_dislike_resp'] = index_dislike_resp
    return row

In [ ]:
test_group['len'].describe()

In [ ]:
train_group['len'].describe()

In [ ]:
test_group = test_group.swifter.apply(split_prompt_resp, axis=1, resp_frac=resp_frac)
train_group = train_group.swifter.apply(split_prompt_resp, axis=1, resp_frac=resp_frac)

In [ ]:
test_group.head(2)

In [ ]:
test_group.shape

In [ ]:
train_group.head(2)

In [ ]:
train_group.shape

In [19]:
if dataset=='movielens':
    def fix_title(title):
        if ", The (" in title:
            name_film, _, year = title.rpartition(", The (")
            title = "The " + name_film + " (" + year
            return title
        if ", A (" in title:
            name_film, _, year = title.rpartition(", A (")
            title = "A " + name_film + " (" + year
        return title
    df_movies = pd.read_csv(os.path.join(datapath, r"movies.dat"), sep="::", names=["item_id", "name", "geners"], encoding='ISO-8859-1')
    df_movies['name'] = df_movies['name'].apply(lambda x: fix_title(x)[0])
    #df_movies['year'] = df_movies['name'].apply(lambda x: fix_title(x)[1])
    import re

    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_entities.tsv'), \
            names=['url', 'id_set'], sep='\t')
    dbpedia_mapping = pd.read_csv(os.path.join(datapath, 'MappingMovielens2DBpedia-1.2.tsv'), \
            names=['id_movie', 'name', 'dbpedia_url'], sep='\t')
    df_movies = dbpedia_mapping.set_index('dbpedia_url').join(df_relations.set_index('url')).reset_index()
    df_movies['name'] = df_movies['name'].apply(lambda x: re.sub("\s+\(\d+\)$", "", x))
    df_movies['name'] = df_movies['name'].apply(lambda x: re.sub(r"[^a-zA-Z0-9]+", ' ', x).strip())
    df_movies.dropna(inplace=True)
    df_movies['id_set'] = df_movies['id_set'].astype(int)
    mapping_dict = df_movies.loc[:, ['id_set', 'name']].set_index('id_set').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['rel', 'id'], sep='\t')

In [20]:
if dataset=='dbbook':
    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_entities.tsv'), sep='\t')
    df_relations['name'] = df_relations['uri'].apply(lambda x: x.split(";")[0])
    temp = df_relations[df_relations['id'].isin(df['item'].unique())]
    mapping_dict = temp.set_index('id').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['id', 'rel'], sep='\t')

In [21]:
if dataset=='lastfm':
    df_relations = pd.read_csv(os.path.join(datapath, 'mapping_items.tsv'), \
            names=['id', 'name'], sep='\t')
    mapping_dict = df_relations.set_index('id').to_dict()['name']
    property_mapping = pd.read_json(os.path.join(datapath, 'entities_names.json'), typ='series')
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['rel', 'id'], sep='\t')

In [ ]:
if dataset=='boardgamegeek':
    df_relations = pd.read_csv(os.path.join(datapath, 'mapped_entities.csv'), sep=';')
    df_relations['name'] = df_relations['primary']
    mapping_dict = df_relations.set_index('id').to_dict()['name']
    property_mapping = pd.read_csv(os.path.join(datapath, 'mapped_prop.csv'), sep=';;', names=['id', 'name'])
    property_mapping = property_mapping.set_index('id').to_dict()['name'] 
    property_mapping = pd.Series(property_mapping) 
    mapping_relation = pd.read_csv(os.path.join(datapath, 'mapped_rel.csv'), \
            names=['id', 'rel'], sep=';;', index_col=False)

In [23]:
import re 

def parse_name_from_url(url):
    name = url.split("/")[-1]
    name = name.replace("_", " ")
    name = name.split(':')[-1]
    name = re.sub(r"\([^()]*\)", "", name)
    pattern = r",.*$"
    name = re.sub(pattern, "", name)
    name = name[0].upper() + name[1:]
    name = " ".join(re.findall('[A-Z][a-z]*', name))
    return name.strip().capitalize()

mapping_relation['name_rel'] = mapping_relation['rel'].apply(parse_name_from_url)
like_dislike_ids = mapping_relation[mapping_relation['name_rel'].isin(['Like', 'Dislike'])]['id'].tolist()
files_prop = [x for x in os.listdir(os.path.join(datapath, 'item-prop')) if 'tsv' in x]
property_graph = pd.concat([pd.read_csv(os.path.join(datapath, 'item-prop', f), \
            names=['item','prop','rel'], sep='\t') for f in files_prop])
property_graph = property_graph[~property_graph['rel'].isin(like_dislike_ids)]
if dataset == 'lastfm':
    property_graph.rename(columns={
        'prop': 'rel', 'rel': 'prop'
    }, inplace=True)
property_graph = property_graph[property_graph['prop'].isin(property_mapping.index)]

In [24]:
lessicalized_prop = {}
if dataset=='dbbook':
    lessicalized_prop['Author'] = 'The book was written by [Author].'
    lessicalized_prop['Series'] =  'This book is part of a series called [Series].'
    lessicalized_prop['Genre'] =  'This is a [Genre] novel.'
    lessicalized_prop['Publisher'] = 'This book was published by [Publisher].'
    lessicalized_prop['Preceded by'] = 'This book is preceded by [Preceded by].'
    lessicalized_prop['Subject'] = 'The story is centered around the theme of [Subject].'
    lessicalized_prop['Illustrator'] = 'The illustrations were done by [Illustrator].'
    lessicalized_prop['Writers'] = 'The screenplay was written by [Writers].'
    lessicalized_prop['Artists'] = 'The artwork was created by [Artists].'
    lessicalized_prop['Letterers'] = 'The lettering was done by [Letterers].'
    lessicalized_prop['Director'] = 'The film adaptation was directed by [Director].'
    lessicalized_prop['Screenplay'] = 'The screenplay was written by [Screenplay].'
    lessicalized_prop['Starring'] = 'The film stars [Starring].'
    lessicalized_prop['Cinematography'] = 'The cinematography was handled by [Cinematography].'
    lessicalized_prop['Producer'] = 'The film was produced by [Producer].'
    lessicalized_prop['Distributor'] = 'The film was distributed by [Distributor].'
    lessicalized_prop['Editor'] = 'This book was edited by [Editor].'
    lessicalized_prop['Category'] = 'This book falls under the category of [Category].'
    lessicalized_prop['Company'] = 'This book was published by [Company].'
    lessicalized_prop['Writer'] = 'This book was written by [Writer].'
    lessicalized_prop['Place'] = 'The story takes place in [Place].'
    lessicalized_prop['Characters'] = 'The main characters include [Characters].'
    lessicalized_prop['Setting'] = 'The story is set in [Setting].'
    lessicalized_prop['Film adaption'] = 'This book has been adapted into a film titled [Film adaption].'
    lessicalized_prop['Political'] = 'This book explores themes of [Political] politics.'
    lessicalized_prop['Title'] = 'The title of the book is [Title].'
    lessicalized_prop['Editors'] = 'This book was edited by [Editors].'
    lessicalized_prop['Creators'] = 'This book was created by [Creators].'
    lessicalized_prop['Magazine'] = 'This story was originally serialized in [Magazine].'
    lessicalized_prop['Studio'] = 'The film adaptation was produced by [Studio].'
    lessicalized_prop['Network'] = 'This book has been adapted into a TV series on [Network].'
    lessicalized_prop['Narrated'] = 'The audiobook is narrated by [Narrated].'
    lessicalized_prop['Creator'] = 'This work was created by [Creator].'
    lessicalized_prop['Published'] = 'The book was published in [Published].'
    lessicalized_prop['Authors'] = 'This book was written by the following authors: [Authors].'
    lessicalized_prop['Co author'] = 'The author co-authored this book with [Co-author name].'

if dataset == 'lastfm':
    lessicalized_prop['Subject'] = 'The artist\'s work focuses primarily on themes of [Subject].'
    lessicalized_prop['Genre'] = 'The artist is known for their unique contribution to the [Genre] genre.'
    lessicalized_prop['Current members'] = 'Currently, the artist\'s lineup includes [Current members].'
    lessicalized_prop['Origin'] = 'The artist hails from [Origin], bringing a distinctive sound to their music.'
    lessicalized_prop['Past members'] = 'Over the years, the artist has seen several changes, with past members including [Past members].'
    lessicalized_prop['Occupation'] = 'In addition to being a musician, the artist also works as [Occupation].'
    lessicalized_prop['Instrument'] = 'The artist is proficient in playing the [Instrument].'
    lessicalized_prop['Genres'] = 'Their music spans across multiple genres, including [Genres].'
    lessicalized_prop['Home town'] = 'The artist grew up in [Home town], which has greatly influenced their music.'
    lessicalized_prop['Religion'] = 'The artist\'s work is often inspired by their religious beliefs in [Religion].'
    lessicalized_prop['Partner'] = 'In their personal life, the artist is partnered with [Partner].'
    lessicalized_prop['Alias'] = 'The artist is also known by the alias [Alias].'
    lessicalized_prop['Title'] = 'The artist holds the title of [Title] in the music industry.'
    lessicalized_prop['Author'] = 'They are also an accomplished author, having written [Author].'
    lessicalized_prop['Work'] = 'The artist\'s notable works include [Work].'
    lessicalized_prop['Notable instruments'] = 'They are renowned for their skill with notable instruments such as [Notable instruments].'
    lessicalized_prop['Manager'] = 'The artist is managed by [Manager].'
    lessicalized_prop['Former members'] = 'Former members of the artist\'s ensemble include [Former members].'
    lessicalized_prop['Voice type'] = 'The artist is recognized for their [Voice type] voice.'
    lessicalized_prop['Siblings'] = 'The artist has [Siblings] siblings who have also influenced their music career.'
    lessicalized_prop['Nationality'] = 'They proudly represent their [Nationality] heritage in their music.'
    lessicalized_prop['Instruments'] = 'The artist is adept at playing several instruments, including [Instruments].'
    lessicalized_prop['Television'] = 'In addition to music, the artist has appeared on television shows such as [Television].'
    lessicalized_prop['Influences'] = 'Their musical influences include [Influences].'
    lessicalized_prop['Agency'] = 'The artist is represented by the agency [Agency].'
    lessicalized_prop['Record labels'] = 'They have been signed with record labels such as [Record labels].'
    lessicalized_prop['Touring members'] = 'When touring, the artist is accompanied by members such as [Touring members].'
    lessicalized_prop['Former labels'] = 'The artist has previously been associated with former labels like [Former labels].'
    lessicalized_prop['Trainer'] = 'Their skills have been honed under the guidance of their trainer, [Trainer].'

if dataset=='movielens':
    lessicalized_prop['Starring'] = 'In this movie, [Starring]  take on a prominent role.'
    lessicalized_prop['Producer'] = 'Producing this movie is [Producer].'
    lessicalized_prop['Language'] = 'This movie is primarily in [Language].'
    lessicalized_prop['Editing'] = 'The editing in this movie is handled by [Editing].'
    lessicalized_prop['Country'] = 'This movie is set in [Country].'
    lessicalized_prop['Music composer'] = 'The musics of this movie is composed by [Music composer].'
    lessicalized_prop['Based on'] = 'This movie is based on [Based on].'
    lessicalized_prop['Subject'] = 'this movie explores [Subject] themes.'
    lessicalized_prop['Cinematography'] = 'The visuals of this movie are captured by [Cinematography].'
    lessicalized_prop['Writer'] = 'The narrative of this movie is penned by [Writer].'
    lessicalized_prop['Director'] = 'Guiding the vision of this movie is the director, [Director].'

if dataset=='boardgamegeek':
    lessicalized_prop['Type'] = 'This game falls under the [Type] category.'
    lessicalized_prop['Primary'] = 'The primary feature of this game is [Primary].'
    lessicalized_prop['Alternate'] = 'An alternate name for this game is [Alternate].'
    lessicalized_prop['Yearpublished'] = 'This game was published in the year [Yearpublished].'
    lessicalized_prop['Minplayers'] = 'The minimum number of players required is [Minplayers].'
    lessicalized_prop['Maxplayers'] = 'The maximum number of players allowed is [Maxplayers].'
    lessicalized_prop['Playingtime'] = 'The average playing time for this game is [Playingtime] minutes.'
    lessicalized_prop['Minplaytime'] = 'The minimum playtime for this game is [Minplaytime] minutes.'
    lessicalized_prop['Maxplaytime'] = 'The maximum playtime for this game is [Maxplaytime] minutes.'
    lessicalized_prop['Minage'] = 'The minimum age recommended to play this game is [Minage] years.'
    lessicalized_prop['Boardgamecategory'] = 'This game belongs to the [Boardgamecategory] category.'
    lessicalized_prop['Boardgamemechanic'] = 'The main mechanic of this game is [Boardgamemechanic].'
    lessicalized_prop['Boardgamefamily'] = 'This game is part of the [Boardgamefamily] family.'
    lessicalized_prop['Boardgameexpansion'] = '[Boardgameexpansion] is an expansion for this game.'
    lessicalized_prop['Boardgameimplementation'] = 'This game implements [Boardgameimplementation].'
    lessicalized_prop['Boardgamedesigner'] = 'The designer of this game is [Boardgamedesigner].'
    lessicalized_prop['Boardgameartist'] = 'The artist for this game is [Boardgameartist].'
    lessicalized_prop['Boardgamepublisher'] = 'This game is published by [Boardgamepublisher].'


In [25]:
mapping_name_rel =property_graph.set_index('rel').join(mapping_relation.loc[:, ['name_rel', 'id']].set_index('id')).reset_index()
goupd_mapping = mapping_name_rel.groupby(['item', 'name_rel']).agg({
    'prop':list
})

In [26]:
temp = goupd_mapping.reset_index()
kn_lex = {}
property_mapping_dict = property_mapping.to_dict()
for item in temp['item'].unique():
    item_df = temp[temp['item'] == item].sort_values('name_rel')
    item_name = mapping_dict.get(item, None)
    if item_name == None:
        continue
    prompt = f'{item_name} is a {items}.'
    for _, row in item_df.iterrows():
        names_property = [property_mapping_dict[x] for x in row['prop'] if property_mapping_dict.get(x, None)!=None]
        prompt += f" {lessicalized_prop[row['name_rel']]}"
        if len(names_property) > 1:
            names_property_str = ", ".join(names_property)
        else:
            names_property_str = names_property[0]
        prompt = prompt.replace(f"[{row['name_rel']}]", names_property_str)
    kn_lex[item] = prompt.strip()

In [27]:
import json

with open(os.path.join(datapath, f'{dataset}_domain_kn_graph.jsonl'), 'w') as outfile:
    for new_id, desc in kn_lex.items():
        json.dump({'target_id':str(new_id), 'text':desc}, outfile)
        outfile.write('\n')

In [28]:
item_desc_df = pd.read_json(os.path.join(datapath, f'{dataset}_domain_kn_graph.jsonl'), lines=True) 
item_desc_df['target_id'] = item_desc_df['target_id'].astype(int)
item_desc_dict = item_desc_df.set_index('target_id').to_dict()['text']

## Prepare prompt

In [ ]:
train_group.head()

In [ ]:
mapping_dict[8670]

In [31]:
def map_with_names(row, col, mapping_dict):
    return [mapping_dict.get(x, '') for x in np.array(row['item'])[row[col]] if len(mapping_dict.get(x, ''))>2]


train_group['liked_prompt'] = train_group.apply(map_with_names, axis=1, col='index_like_prompt', mapping_dict=mapping_dict)
train_group['liked_resp'] = train_group.apply(map_with_names, axis=1, col='index_like_resp', mapping_dict=mapping_dict)
train_group['disliked_prompt'] = train_group.apply(map_with_names, axis=1, col='index_dislike_prompt', mapping_dict=mapping_dict)
train_group['disliked_resp'] = train_group.apply(map_with_names, axis=1, col='index_dislike_resp', mapping_dict=mapping_dict)

test_group['liked_prompt'] = test_group.apply(map_with_names, axis=1, col='index_like_prompt', mapping_dict=mapping_dict)
test_group['liked_resp'] = test_group.apply(map_with_names, axis=1, col='index_like_resp', mapping_dict=mapping_dict)
test_group['disliked_prompt'] = test_group.apply(map_with_names, axis=1, col='index_dislike_prompt', mapping_dict=mapping_dict)
test_group['disliked_resp'] = test_group.apply(map_with_names, axis=1, col='index_dislike_resp', mapping_dict=mapping_dict)

In [ ]:
train_group.head(2)

In [33]:
train = []
test = []
for _, row in test_group.iterrows():
    for item in row['liked_prompt']:
        train.append({
            'user_id': row['user'],
            'item_id': item,
            'score': 1
        })
    for item in row['disliked_prompt']:
        train.append({
            'user_id': row['user'],
            'item_id': item,
            'score': 0
        })
    for item in row['liked_resp']:
        test.append({
            'user_id': row['user'],
            'item_id': item,
            'score': 1
        })
    for item in row['disliked_resp']:
        test.append({
            'user_id': row['user'],
            'item_id': item,
            'score': 0
        })

pd.DataFrame(train).to_csv("train_adaptation.csv")
                    

In [34]:
import random
import itertools

def prepare_user_prompt(row):
    random.seed(22)
    user_prompt = ''
    liked_list = ', '
    disliked_list = ', '
    liked_part = """I like the following items:\n{liked_list}\n"""
    disliked_part = """I dislike the following items:\n{disliked_list}\n"""
    liked_items = sorted(row['liked_prompt'], key=lambda k: random.random())
    disliked_items = sorted(row['disliked_prompt'], key=lambda k: random.random())
    liked_list = liked_list.join(liked_items)
    disliked_list = disliked_list.join(disliked_items)
    if len(liked_items)>0:
        user_prompt = user_prompt + liked_part.format(liked_list=liked_list)
    if len(disliked_list)>0:
        user_prompt = user_prompt + disliked_part.format(disliked_list=disliked_list)
    
    to_rank_list_str = ', '
    response_part = """\nRank the following items:\n{to_rank_list}"""
    to_rank_list = list(itertools.chain(*[row['liked_resp'], row['disliked_resp']]))
    to_rank_list = sorted(to_rank_list, key=lambda k: random.random())
    to_rank_list = to_rank_list_str.join(to_rank_list)
    user_prompt = user_prompt + response_part.format(to_rank_list=to_rank_list)
    return user_prompt
        
def get_assistant_prompt(row):
    ranked_list_str = ''
    rank_part = """Here a list:\n{ranked_list}\n"""
    for i, x in enumerate(row['liked_resp']):
        ranked_list_str = ranked_list_str + f"{i+1}. {x}\n"
    for i, x in enumerate(row['disliked_resp'], len(row['liked_resp'])):
        ranked_list_str = ranked_list_str + f"{i+1}. {x}\n"
    return rank_part.format(ranked_list=ranked_list_str)[:-1]


train_group['user_prompt'] = train_group.apply(prepare_user_prompt, axis=1)
train_group['assistant_prompt'] = train_group.apply(get_assistant_prompt, axis=1)

test_group['user_prompt'] = test_group.apply(prepare_user_prompt, axis=1)
test_group['assistant_prompt'] = test_group.apply(get_assistant_prompt, axis=1)

In [ ]:
print(train_group.loc[150, 'user_prompt'])

In [ ]:
print(train_group.loc[150, 'assistant_prompt'])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../llama3")
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template =  "{% set ns = namespace(i=0) %}" \
#                            "{% for message in messages %}" \
#                                "{% if message['role'] == 'system' and ns.i == 0 %}" \
#                                       "{{ bos_token +' [INST] <<SYS>>\n' }}" \
#                                       "{{ message['content'] + ' <</SYS>>\n'}}" \
#                                "{% elif message['role'] == 'user' %}" \
#                                    "{{ message['content'] + ' [/INST]\n'}}" \
#                                "<|reserved_special_token_22|>\n"\
#                                "{% elif message['role'] == 'assistant' %}" \
#                                    "{{ message['content'] + ' ' + eos_token }}" \
#                                "{% endif %}" \
#                                "{% set ns.i = ns.i+1 %}" \
#                            "{% endfor %}"
#
tokenizer.chat_template =  "{% set ns = namespace(i=0) %}" \
                            "{% for message in messages %}" \
                                "{% if message['role'] == 'system' and ns.i == 0 %}" \
                                       "{{ bos_token +' [INST] <<SYS>>\n' }}" \
                                       "{{ message['content'] + ' <</SYS>>\n'}}" \
                                "{% elif message['role'] == 'user' %}" \
                                    "{{ message['content'] + ' [/INST]\n'}}" \
                                "{% elif message['role'] == 'assistant' %}" \
                                    "{{ message['content'] + '' + eos_token }}" \
                                "{% endif %}" \
                                "{% set ns.i = ns.i+1 %}" \
                            "{% endfor %}"

In [38]:
def get_prompt(row, is_train=True):
    messages = [
    {"role": "system", "content": f"You're operating as a {domain} recommendation system. Your task involves receiving a user's list of liked and disliked items, along with a set of candidate items, and then reordering them based on preferences."},
    {"role": "user", "content": row['user_prompt']}
    ]
    if is_train:
        messages.append({"role": "assistant", "content": row['assistant_prompt']})
    return tokenizer.apply_chat_template(messages, tokenize=False)

In [39]:
train_group['prompt'] = train_group.apply(get_prompt, axis=1, is_train=True)
test_group['prompt'] = test_group.apply(get_prompt, axis=1, is_train=False)

In [ ]:
train_group.head(2)

In [41]:
def get_description_liked(row):
    liked_item_index = row['index_like_resp']
    likex_items = np.array(row['item'])[liked_item_index]
    return '\n'.join([item_desc_dict.get(y, '') for y in likex_items])

In [42]:
train_group['descriptions'] = train_group['item'].apply(lambda x: [item_desc_dict.get(y, '') for y in x if len(item_desc_dict.get(y, ''))>2])
#train_group['descriptions'] = train_group.apply(get_description_liked, axis=1)

In [ ]:
train_group.head(2)

In [44]:
train_group.to_pickle(os.path.join(datapath, f'{dataset}_train_dataset_graph.pkl'))
test_group.to_pickle(os.path.join(datapath, f'{dataset}_test_dataset_graph.pkl'))

In [45]:
prompts = train_group['prompt'].tolist() #* 3

In [46]:
descriptions = list(item_desc_dict.values())

In [47]:
descriptions = [f'{tokenizer.bos_token} ' + x + f' {tokenizer.eos_token}' for x in descriptions]

In [48]:
import json

with open(os.path.join(datapath, f'{dataset}_graph_kn_train_set.jsonl'), 'w') as outfile:
    for x in descriptions:
        json.dump({'text':x}, outfile)
        outfile.write('\n')

In [48]:
import random 

random.seed(22)
train_set = prompts + descriptions* 3

random.shuffle(train_set)

In [ ]:
len(train_set)

In [50]:
import json

with open(os.path.join(datapath, f'{dataset}_train_set_kgraph.jsonl'), 'w') as outfile:
    for x in train_set:
        json.dump({'text':x}, outfile)
        outfile.write('\n')

In [51]:
import json

with open(os.path.join(datapath, f'{dataset}_train_set_no_kn.jsonl'), 'w') as outfile:
    for x in train_group['prompt'].tolist():
        json.dump({'text':x}, outfile)
        outfile.write('\n')

In [53]:
prompts = test_group['prompt'].tolist()

In [54]:
import json

with open(os.path.join(datapath, f'{dataset}_test_set.jsonl'), 'w') as outfile:
    for x in prompts:
        json.dump({'text':x}, outfile)
        outfile.write('\n')

# Baseline

In [49]:
train_bs = pd.concat([test_group.loc[:, ['user', 'index_like_prompt', 'index_dislike_prompt', 'item']],\
                      train_group.loc[:, ['user', 'index_like_prompt', 'index_dislike_prompt', 'item']]])
test_bs = test_group.loc[:, ['user', 'index_like_resp', 'index_dislike_resp', 'item']]

In [50]:
train_bs['bs_format'] = train_bs.apply(lambda x:[{'user_id:token': x['user'], 'item_id:token': x['item'][item], 'rating:float': 1} for item in x['index_like_prompt']], axis=1)
train_bs['bs_format'] = train_bs['bs_format'] + train_bs.apply(lambda x:[{'user_id:token': x['user'], 'item_id:token': x['item'][item], 'rating:float': 0} for item in x['index_dislike_prompt']], axis=1)

test_bs['bs_format'] = test_bs.apply(lambda x:[{'user_id:token': x['user'], 'item_id:token': x['item'][item], 'rating:float': 1} for item in x['index_like_resp']], axis=1)
test_bs['bs_format'] = test_bs['bs_format'] + test_bs.apply(lambda x:[{'user_id:token': x['user'], 'item_id:token': x['item'][item], 'rating:float': 0} for item in x['index_dislike_resp']], axis=1)

In [51]:
item_set_bs = set(x['item_id:token'] for x in list(itertools.chain(*test_bs['bs_format'].tolist())) + list(itertools.chain(*train_bs['bs_format'].tolist())))

In [52]:
property_graph_bs = property_graph[property_graph['item'].isin(item_set_bs)].rename(columns={
    'item':"head_id:token",
    'rel': "relation_id:token",
    'prop': "tail_id:token"
})
link_bs = property_graph_bs.loc[:, ['head_id:token']].copy()
link_bs['item_id:token'] = link_bs.loc[:, ['head_id:token']].apply(lambda x:x)
link_bs = link_bs.rename(columns={
    'head_id:token': "entity_id:token",
})

property_graph_bs.loc[:, ['head_id:token', 'relation_id:token', 'tail_id:token']].to_csv(os.path.join(datapath, 'baseline', f'{dataset}.kg'), sep='\t', index=False)
link_bs.to_csv(os.path.join(datapath, 'baseline', f'{dataset}.link'), sep='\t', index=False)

In [62]:
import itertools

os.makedirs(os.path.join(datapath, 'baseline'), exist_ok=True)
pd.DataFrame(list(itertools.chain(*test_bs['bs_format'].tolist()))).to_csv(os.path.join(datapath, 'baseline', f'{dataset}.part3.inter'), sep='\t', index=False)
pd.DataFrame(list(itertools.chain(*train_bs['bs_format'].tolist()))).to_csv(os.path.join(datapath, 'baseline', f'{dataset}.part1.inter'), sep='\t', index=False)

# EXTRA

In [264]:
splits = np.array_split(train_group['descriptions'].to_list(), 10)

In [265]:
lens = []
for split in splits:
    tok = tokenizer(split.tolist(), return_length=True)
    lens.append(tok['length'])

In [ ]:
display(pd.Series(list(chain(*lens))).describe())

In [ ]:
tokenizer.encode("<|reserved_special_token_22|>", add_special_tokens=False)

In [ ]:
if dataset in ['dbbook', 'movielens']:
    item_prop_df = pd.read_csv(os.path.join(datapath, 'item-prop', 'train.tsv'), \
                names=['item', 'prop', 'rel'], sep='\t')
else:
    item_prop_df = pd.read_csv(os.path.join(datapath, 'item-prop', 'train.tsv'), \
                names=['item', 'rel', 'prop'], sep='\t')

In [ ]:
item_prop_df

In [ ]:
if dataset == 'dbbook':
    mapping_rel = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
            names=['id_rel', 'name_rel'], sep='\t')
else:
    mapping_rel = pd.read_csv(os.path.join(datapath, 'mapping_relations.tsv'), \
                names=['name_rel', 'id_rel'], sep='\t')
mapping_rel.head()

In [ ]:
mapping_rel['name_rel_red']=mapping_rel['name_rel'].apply(lambda x: x.split("/")[-1])

In [ ]:
mapping_rel

In [ ]:
mapping_rel[mapping_rel['id_rel'].isin(item_prop_df['rel'].unique())]

In [ ]:
item_desription = pd.read_csv(os.path.join(datapath, dataset+'.txt'), sep=';;', names=['item_id', 'description'],  on_bad_lines='skip')

In [ ]:
item_desription.head()

In [ ]:
len(item_desription['description'].unique())

In [ ]:
item_desription.shape

In [ ]:
len(item_desription[item_desription['item_id'].isin(user_item_df['item'].unique())]['description'].unique())